In [33]:
import openai
import json
import os
import dotenv
import datetime

In [34]:
dotenv.load_dotenv(".env", override=True)
OPENAI_API_KEY = os.environ.get("OPENAI_API_KEY")
assert OPENAI_API_KEY is not None
openai.api_key = OPENAI_API_KEY

In [35]:
# Define the functions
def getproductavailabilitydatebyzip(zip, productcode, deliveryservicedays):
    # Your implementation here
    return {
        'availability_date': '2023-07-20'
    }

def getproductinventorybyloc(loc):
    # Your implementation here
    return {
        'inventory_details': [
            {'loc_id': 1, 'inventory': 100, 'availability': 80},
            {'loc_id': 2, 'inventory': 200, 'availability': 180}
        ]
    }

def getproductprice(zip, customerid, productcode):
    # Your implementation here
    return {
        'price': 199.99
    }

In [36]:
# Map the function names to the functions
available_functions = {
    "getproductavailabilitydatebyzip": getproductavailabilitydatebyzip,
    "getproductinventorybyloc": getproductinventorybyloc,
    "getproductprice": getproductprice,
}

In [37]:
def run_conversation():
    # Define the system message
    system_message = "You are a helpful Customer Service chatbot answering questions about products, especially delivery times, product availability, and product price."

    # Define the function descriptions for the API
    functions = [
        {
            "name": "getproductavailabilitydatebyzip",
            "func": getproductavailabilitydatebyzip,
            "schema": {
                "type": "object",
                "properties": {
                    "zip": {"type": "string"},
                    "productcode": {"type": "string"},
                    "deliveryservicedays": {"type": "integer"}
                },
                "required": ["zip", "productcode", "deliveryservicedays"],
                "output": {
                    "type": "object",
                    "properties": {
                        "availability_date": {"type": "string", "format": "date"}
                    }
                }
            }
        },
        {
            "name": "getproductinventorybyloc",
            "func": getproductinventorybyloc,
            "schema": {
                "type": "object",
                "properties": {
                    "loc": {"type": "string"}
                },
                "required": ["loc"],
                "output": {
                    "type": "object",
                    "properties": {
                        "inventory_details": {"type": "array",
                                            "items": {
                                                "type": "object",
                                                "properties": {
                                                    "loc_id": {"type": "integer"},
                                                    "inventory": {"type": "integer"},
                                                    "availability": {"type": "integer"}
                                                },
                                                "required": ["loc_id", "inventory", "availability"]
                                            }
                                            }
                    }
                }
            }
        },
        {
            "name": "getproductprice",
            "func": getproductprice,
            "schema": {
                "type": "object",
                "properties": {
                    "zip": {"type": "string"},
                    "customerid": {"type": "string"},
                    "productcode": {"type": "string"}
                },
                "required": ["zip", "customerid", "productcode"],
                "output": {
                    "type": "object",
                    "properties": {
                        "price": {"type": "number"}
                    }
                }
            }
        }
    ]


    # Start the conversation with the user's question
    user_question = input("Ask your question: ")
    messages = [{"role": "system", "content": system_message}, {"role": "user", "content": user_question}]

    # Call the model
    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo-0613",
        messages=messages,
        functions=functions,
        function_call="auto",
    )
    response_message = response["choices"][0]["message"]
    print(response_message)
    # Check if GPT wanted to call a function
    if response_message.get("function_call"):
        function_name = response_message["function_call"]["name"]
        fuction_to_call = available_functions[function_name]
        function_args = json.loads(response_message["function_call"]["arguments"])
        function_response = fuction_to_call(**function_args)

        # Send the info on the function call and function response to GPT
        messages.append(response_message)
        messages.append(
            {
                "role": "function",
                "name": function_name,
                "content": function_response,
            }
        )
        second_response = openai.ChatCompletion.create(
            model="gpt-3.5-turbo-0613",
            messages=messages,
        )
        return second_response
    
# Run the conversation
print(run_conversation())

TypeError: Object of type function is not JSON serializable